# My Spotify Analytics Project

## Loading libraries

In [3]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3
import streamlit as st

In [4]:
# Writing down some constants.

In [5]:
DATABASE_LOCATION = "#" # Sorry, I decided to hide this for privacy reason!
USER_ID = "#" # Sorry, I decided to hide this for privacy reason!
TOKEN = "#" # Sorry, I decided to hide this for privacy reason!

## Extract

In [6]:
if __name__ == "__main__":
    headers = {
        "Acept": "application/json",
        "Content-Type": "application/json",
        "Authorization": "Bearer {token}" .format(token=TOKEN)
    }
    
    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    yesterday_unix_timestap = int(yesterday.timestamp()) * 1000
    
    # API Request
    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}" .format(time=yesterday_unix_timestap), headers = headers)
    data = r.json()
    
    print(data)


{'items': [{'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/23TFHmajVfBtlRx5MXqgoz'}, 'href': 'https://api.spotify.com/v1/artists/23TFHmajVfBtlRx5MXqgoz', 'id': '23TFHmajVfBtlRx5MXqgoz', 'name': 'Sfera Ebbasta', 'type': 'artist', 'uri': 'spotify:artist:23TFHmajVfBtlRx5MXqgoz'}], 'available_markets': ['AD', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'EC', 'EE', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KZ', 'LA', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK', 'ML', 'MN', 'MO', 'MR', 'MT', 'MU', 'MV'

In [23]:
# Trying to catch clean data.
for song in data['items']:
    print(len(song['track']['available_markets']))

169
182
182
182
182
178
183
183
183
182
183
182
182
178
183
182
178
182
183
183


In [24]:
# It works! Let's do a for cycle in order to get clean data for the Pandas DataFrame!

# Creating lists for the dict.
song_names = []
artist_names = []
popularity_list = []
available_markets_list = []
played_at_list = []
timestamps = []


for song in data['items']:
    song_names.append(song['track']['name'])
    popularity_list.append(song['track']['popularity'])
    available_markets_list.append(len(song['track']['available_markets']))
    artist_names.append(song['track']['artists'][0]['name'])
    played_at_list.append(song["played_at"])
    timestamps.append(song['played_at'][0:10])

#Creating the dict: I will use it in order to build the DataFrame.
song_dict = {
    'song_name': song_names,
    'artist_name': artist_names,
    'played_at': played_at_list,
    'timestamp': timestamps,
    'popularity': popularity_list,
    'available_markets': available_markets_list
}

song_df = pd.DataFrame(data=song_dict, columns=['song_name', 'popularity', 'available_markets', 'artist_name', 'played_at', 'timestamp'])

#Trying to visualize the DataFrame.
song_df

,song_name,popularity,available_markets,artist_name,played_at,timestamp
0,BRNBQ,59,169,Sfera Ebbasta,2022-05-11T21:47:01.859Z,2022-05-11
1,Wow.,77,182,Post Malone,2022-05-11T21:42:24.847Z,2022-05-11
2,3 PALI,65,182,Lazza,2022-05-11T21:39:54.415Z,2022-05-11
3,Myself,61,182,Post Malone,2022-05-11T21:36:41.483Z,2022-05-11
4,NESSUNO (feat. Geolier),68,182,Lazza,2022-05-11T21:34:02.328Z,2022-05-11
5,I Love It (& Lil Pump),70,178,Kanye West,2022-05-11T21:31:22.320Z,2022-05-11
6,Tell The Vision,53,183,Kanye West,2022-05-11T21:29:14.329Z,2022-05-11
7,No Option,66,183,Post Malone,2022-05-11T21:27:22.241Z,2022-05-11
8,all I know (feat. Trippie Redd),61,183,Machine Gun Kelly,2022-05-11T21:24:19.563Z,2022-05-11
9,make up sex (feat. blackbear),78,182,Machine Gun Kelly,2022-05-11T21:22:10.064Z,2022-05-11


In [25]:
#Saving the CSV file, I will use it for the Streamlit app!
song_df.to_csv('song_df.csv', index=False)